<a href="https://colab.research.google.com/github/Theophilus-Baidoo/Data--Driven-Survival-Modeling-for-Breast-Cancer/blob/main/Breast_CancerDeepSurv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sklearn_pandas
!pip install torchtuples
!pip install pycox
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchtuples as tt
from torch import nn
import torch.nn.functional as F
from pycox.models import CoxPH
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv
from pycox.preprocessing.feature_transforms import OrderedCategoricalLong
from pycox.datasets import support
from pycox.models.loss import rank_loss_deephit_single
from sklearn.preprocessing import OneHotEncoder
import optuna
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
import torchtuples as tt
from pycox.models import CoxPH
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
import pandas as pd

mydata1 = pd.read_csv("C:/Users/gyedu/OneDrive/Desktop/SEER Breast Cancer Dataset .csv")

In [ ]:
# rename some of the column
mydata1 = mydata1.rename(columns={'Race ': 'Race'})
mydata1 = mydata1.rename(columns={'T Stage ': 'TStage'})
mydata1= mydata1.rename(columns={'N Stage': 'NStage'})
mydata1 = mydata1.rename(columns={'Marital Status': 'Marital'})
mydata1= mydata1.rename(columns={'6th Stage': '6thStage'})
mydata1 = mydata1.rename(columns={'A Stage': 'AStage'})
mydata1 = mydata1.rename(columns={'Estrogen Status': 'EStatus'})
mydata1 = mydata1.rename(columns={'Progesterone Status': 'PStatus'})

In [ ]:
mydata1["Race"].replace("Other (American Indian/AK Native, Asian/Pacific Islander)", "Other", inplace=True)
mydata1["Grade"].replace("Well differentiated; Grade I", "Grade I", inplace=True)
mydata1["Grade"].replace("Moderately differentiated; Grade II", "Grade II", inplace=True)
mydata1["Grade"].replace("Poorly differentiated; Grade III", "Grade III", inplace=True)
mydata1["Grade"].replace("Undifferentiated; anaplastic; Grade IV", "Grade IV", inplace=True)
mydata1["Marital"].replace("Single (never married)", "Single", inplace=True)
mydata1["Marital"].replace("Married (including common law)", "Married", inplace=True)

In [ ]:
mydata1['Status'].replace(['Alive','Dead'],[0,1],inplace=True)
mydata1['TStage'].replace(['T1','T2','T3','T4'],[0,1,2,3],inplace=True)
mydata1['NStage'].replace(['N1','N2','N3'],[0,1,2],inplace=True)
mydata1['Grade'].replace(['Grade I','Grade II','Grade III','Grade IV'],[0,1,2,3],inplace=True)
mydata1['EStatus'].replace(['Positive','Negative'],[1,0],inplace=True)
mydata1['PStatus'].replace(['Positive','Negative'],[1,0],inplace=True)
mydata1['AStage'].replace(['Regional','Distant'],[0,1],inplace=True)

In [ ]:
mydata1 = mydata1.drop('6thStage',axis = 1)

In [ ]:
# Convert categorical variables into numerical ones using one-hot encoding
encoder = OneHotEncoder(handle_unknown='ignore')
cat_cols = ['Race','Marital']
encoded_cols = encoder.fit_transform(mydata1[cat_cols])
encoded_cols_mydata1 = pd.DataFrame(encoded_cols.toarray(), columns=encoder.get_feature_names_out(cat_cols))

In [ ]:
mydata1= mydata1.drop(cat_cols, axis=1)
mydata1 = pd.concat([mydata1, encoded_cols_mydata1], axis=1)

In [ ]:
mydata1_test=mydata1.sample(frac=0.2)
mydata1_train=mydata1.drop(mydata1_test.index)
mydata1_val=mydata1_train.sample(frac=0.2)
mydata1_train=mydata1_train.drop(mydata1_val.index)

In [ ]:
cols_stand = ['Age', 'Tumor Size', 'Regional Node Examined', 'Reginol Node Positive', 'TStage', 'NStage', 'Grade']

cols_leave = ['EStatus','PStatus','AStage','Race_Black','Race_Other','Race_White', 'Marital_Divorced','Marital_Married','Marital_Separated',
              'Marital_Single','Marital_Widowed']

In [ ]:
standardize = [([col], StandardScaler()) for col in cols_stand]
leave = [(col, None) for col in cols_leave]

In [ ]:
x_mapper = DataFrameMapper(standardize + leave)
x_train = x_mapper.fit_transform(mydata1_train).astype('float32')
x_val = x_mapper.transform(mydata1_val).astype('float32')
x_test = x_mapper.transform(mydata1_test).astype('float32')

In [ ]:
get_target = lambda mydata1: (mydata1['Survival Months'].values, mydata1['Status'].values)
y_train = get_target(mydata1_train)
y_val = get_target(mydata1_val)
durations_test, events_test = get_target(mydata1_test)
val = x_val, y_val

In [ ]:
import torch

def create_net(trial, input_features):
    # Define hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 6)
    units = trial.suggest_categorical('units', [32,64, 128])
    dropout = trial.suggest_float('dropout',  0.1, 0.3)
    #lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)

    layers = []
    for i in range(num_layers):
        if i == 0:
            layers.append(torch.nn.Linear(input_features, units))
        else:
            layers.append(torch.nn.Linear(units, units))
        layers.append(torch.nn.ReLU())
        layers.append(torch.nn.Dropout(dropout))

    layers.append(torch.nn.Linear(units, 1))  # Output layer
    return torch.nn.Sequential(*layers)

# Then, use this function in your objective:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    net = create_net(trial, input_features=x_train.shape[1])
    model = CoxPH(net, optimizer=tt.optim.Adam(lr=lr))

    # Fit model
    batch_size = trial.suggest_int('batch_size', 64, 100, log=True)
    model.fit(x_train, y_train, batch_size, epochs=100, callbacks=[tt.callbacks.EarlyStopping()])

    # Evaluate model
    _ = model.compute_baseline_hazards()
    # surv = model.predict_surv_df(x_test)
    surv = model.predict_surv_df(x_val)
    ev = EvalSurv(surv, y_val[0], y_val[1], censor_surv='km')
    c_index = ev.concordance_td()

    return -c_index  # Negative C-index because Optuna minimizes the objective

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best parameters: ", best_params)

In [ ]:
# best_params should contain keys like 'num_layers', 'units', 'dropout', etc.
from sklearn.model_selection import KFold
import numpy as np
import torchtuples as tt
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

best_params = study.best_params

c_indices_all = []
ibs_scores_all = []

for iteration in range(10):
    random_state = 2345 + iteration
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    c_indices = []
    ibs_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_train)):
        x_train_fold = x_train[train_idx]
        y_train_fold = y_train[0][train_idx], y_train[1][train_idx]
        x_val_fold = x_train[val_idx]
        y_val_fold = y_train[0][val_idx], y_train[1][val_idx]

        # Using best parameters for MLPVanilla architecture
        in_features = x_train.shape[1]
        out_features = 1  # assuming CoxPH model
        num_nodes = [best_params['units']] * best_params['num_layers']
        batch_norm = False  # set as needed
        dropout = best_params['dropout']
        output_bias = False  # set as needed

        net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout=dropout, output_bias=output_bias)
        model = CoxPH(net, tt.optim.Adam(lr=best_params['lr']))

        # Train the model
        batch_size =90 #best_params['batch_size']
        epochs = 100  # Adjust as necessary
        callbacks = [tt.callbacks.EarlyStopping()]
        verbose = True

        log = model.fit(x_train_fold, y_train_fold, batch_size, epochs, callbacks, verbose,
                        val_data=(x_val_fold, y_val_fold), val_batch_size=batch_size)
#         log.plot()
        # Evaluate the model
        _ = model.compute_baseline_hazards()
        surv = model.predict_surv_df(x_test)
        # surv = model.predict_surv_df(x_val_fold)
        # ev = EvalSurv(surv, y_val_fold[0], y_val_fold[1], censor_surv='km')
        ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

        # Compute C-index and IBS
        c_index = ev.concordance_td()
        c_indices.append(c_index)
#         time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
        time_grid = np.linspace(y_val_fold[0].min(), y_val_fold[0].max(), 100)
        ibs = ev.integrated_brier_score(time_grid)
        ibs_scores.append(ibs)

    c_indices_all.append(np.mean(c_indices))
    ibs_scores_all.append(np.mean(ibs_scores))

    print(f'Iteration {iteration+1}, Average C-index: {np.mean(c_indices)}, Average IBS: {np.mean(ibs_scores)}')

# Calculate the overall average and standard deviation outside the iterations loop
final_avg_c_index = np.mean(c_indices_all)
std_dev_c_index = np.std(c_indices_all)

final_avg_ibs = np.mean(ibs_scores_all)
std_dev_ibs = np.std(ibs_scores_all)

print(f'Overall Average C-index: {final_avg_c_index}, Standard Deviation of C-index: {std_dev_c_index}')
print(f'Overall Average IBS: {final_avg_ibs}, Standard Deviation of IBS: {std_dev_ibs}')

In [ ]:
# best_params should contain keys like 'num_layers', 'units', 'dropout', etc.
from sklearn.model_selection import KFold
import numpy as np
import torchtuples as tt
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

best_params = study.best_params

c_indices_all = []
ibs_scores_all = []

for iteration in range(10):
    random_state = 2775 + iteration
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    c_indices = []
    ibs_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_train)):
        x_train_fold = x_train[train_idx]
        y_train_fold = y_train[0][train_idx], y_train[1][train_idx]
        x_val_fold = x_train[val_idx]
        y_val_fold = y_train[0][val_idx], y_train[1][val_idx]

        # Using best parameters for MLPVanilla architecture
        in_features = x_train.shape[1]
        out_features = 1  # assuming CoxPH model
        num_nodes = [best_params['units']] * best_params['num_layers']
        batch_norm = False  # set as needed
        dropout = best_params['dropout']
        output_bias = False  # set as needed

        net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout=dropout, output_bias=output_bias)
        model = CoxPH(net, tt.optim.Adam(lr=best_params['lr']))

        # Train the model
        batch_size =91 #best_params['batch_size']
        epochs = 250  # Adjust as necessary
        callbacks = [tt.callbacks.EarlyStopping()]
        verbose = True

        log = model.fit(x_train_fold, y_train_fold, batch_size, epochs, callbacks, verbose,
                        val_data=(x_val_fold, y_val_fold), val_batch_size=batch_size)
#         log.plot()
        # Evaluate the model
        _ = model.compute_baseline_hazards()
        surv = model.predict_surv_df(x_test)
        # surv = model.predict_surv_df(x_val_fold)
        # ev = EvalSurv(surv, y_val_fold[0], y_val_fold[1], censor_surv='km')
        ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

        # Compute C-index and IBS
        c_index = ev.concordance_td()
        c_indices.append(c_index)
#         time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
        time_grid = np.linspace(y_val_fold[0].min(), y_val_fold[0].max(), 100)
        ibs = ev.integrated_brier_score(time_grid)
        ibs_scores.append(ibs)

    c_indices_all.append(np.mean(c_indices))
    ibs_scores_all.append(np.mean(ibs_scores))

    print(f'Iteration {iteration+1}, Average C-index: {np.mean(c_indices)}, Average IBS: {np.mean(ibs_scores)}')

# Calculate the overall average and standard deviation outside the iterations loop
final_avg_c_index = np.mean(c_indices_all)
std_dev_c_index = np.std(c_indices_all)

final_avg_ibs = np.mean(ibs_scores_all)
std_dev_ibs = np.std(ibs_scores_all)

print(f'Overall Average C-index: {final_avg_c_index}, Standard Deviation of C-index: {std_dev_c_index}')
print(f'Overall Average IBS: {final_avg_ibs}, Standard Deviation of IBS: {std_dev_ibs}')

In [ ]:
!pip install shap
import shap

In [ ]:
# Convert your numpy arrays to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)

In [ ]:
background = x_train_tensor  # You can choose a different subset

# Initialize SHAP DeepExplainer
explainer = shap.DeepExplainer(model.net, background)

In [ ]:
# Compute SHAP values for a subset of your test set
shap_values = explainer.shap_values(x_test_tensor)

In [ ]:
#  Visualization
shap.summary_plot(shap_values, features=x_test_tensor.numpy(), feature_names=x_mapper.transformed_names_)

In [ ]:
import shap
import torch


shap.summary_plot(shap_values, features=x_test, feature_names=x_mapper.transformed_names_, plot_type='bar')